In [3]:
#attemmpt to optimize aggregration of aging clocks with weights instead of taking the mean evenly
#regresison on all clocks predictions to assign weights to each clock
#splitting GTEx Data by even distributions of age as well as horvath blood data 

In [1]:
from io import StringIO 
import pandas as pd
import numpy as np
import tensorflow as tf
import time
from collections import Counter
import matplotlib.pyplot as plt
import re
import io
import plotly.graph_objects as go
import base64
import os
import joblib
import csv
from clocks import han2020_coefs
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import math
from app import get_altum_age, get_hannum_age, get_horvath_age, get_pheno_age, get_horvath_sb_age, get_ZhangEn_age, get_Han2020_age, get_YingCaus_age


organs = ['Blood', 'Breast', 'Colon', 'Kidney', 'Lung', 'Muscle', 'Ovary', 'Prostate', 'Testis']

zhangen_params = pd.read_table('clocks/zhangen.coef', sep=' ')
zhangen_params['feature'] = zhangen_params['probe']
zhangen_params['coefficient'] = zhangen_params['coef']
zhangen_cpgs = zhangen_params['feature'][1:].tolist()

yingCausAge_params = pd.read_csv('clocks/YingCausAge.csv')
yingCausAge_params['feature'] = yingCausAge_params['term']
yingCausAge_params['coefficient'] = yingCausAge_params['estimate']
yingCausAge_cpgs = yingCausAge_params['feature'][1:].tolist()

cpgs = pd.read_csv("tools/union_out.csv")
list_cpgs = cpgs.iloc[:, 0].tolist() + yingCausAge_cpgs + zhangen_cpgs + han2020_coefs.cpgs
print(len(list_cpgs))
list_cpgs = list(set(list_cpgs))
print(len(list_cpgs))

2024-07-11 06:18:46.690812: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/yashagarwal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Lasso from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/yashagarwal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.

22783
22616


In [2]:
def get_true_age(df):
    age_data = df.iloc[9, 1:]
    ages = []
    for i in range(len(age_data)):
        ages.append(int(age_data.iloc[i].split('-')[0]) + 5)
    return np.array(ages)
    

anno = pd.read_table("/Users/yashagarwal/Documents/GTEx/GTEx_anno/GTEx_Blood.anno.csv")
#print(get_true_age(anno))


def get_true_age_sample(df):
    age_data = df.iloc[9, 1:]
    ages = []
    columns = age_data.index
    
    for i in range(len(age_data)):
        ages.append(int(age_data.iloc[i].split('-')[0]) + 5)
    
    age_df = pd.DataFrame({'Column': columns, 'Age': np.array(ages)})
    return age_df

print(get_true_age_sample(anno))




                      Column  Age
0   GTEX-117YW-0003-SM-GW185   55
1   GTEX-11DYG-0004-SM-GW225   65
2   GTEX-11EMC-0004-SM-GW1YF   65
3   GTEX-11TT1-0001-SM-GW21F   25
4   GTEX-11ZU8-0002-SM-GW18V   65
5   GTEX-11ZUS-0003-SM-GW1V6   35
6   GTEX-12KS4-0003-SM-GW1BP   35
7   GTEX-12WSA-0001-SM-GW187   65
8   GTEX-12WSE-0002-SM-GW19L   25
9   GTEX-12WSI-0004-SM-GW2CP   55
10  GTEX-12ZZW-0003-SM-GW18R   55
11  GTEX-12ZZY-0004-SM-GW19K   65
12  GTEX-139YR-0004-SM-GW1DM   55
13  GTEX-13NYS-0003-SM-GW1FJ   65
14  GTEX-13NZA-0003-SM-GW1F3   65
15  GTEX-13OW5-0003-SM-GW2B3   65
16  GTEX-13QIC-0004-SM-GW2BL   65
17  GTEX-13SLX-0003-SM-GW18M   55
18  GTEX-14A6H-0003-SM-GW21E   35
19  GTEX-14C39-0004-SM-GW17K   45
20  GTEX-14E6D-0002-SM-GW17I   55
21  GTEX-15CHQ-0002-SM-GW21D   65
22  GTEX-15D1Q-0004-SM-GW17J   45
23  GTEX-15DDE-0001-SM-GW17D   55
24  GTEX-15EU6-0003-SM-GW17V   55
25  GTEX-15SHU-0001-SM-GW17T   45
26  GTEX-1A3MV-0002-SM-GW17U   35
27  GTEX-1B98T-0003-SM-GW1CR   65
28  GTEX-1C64N

In [11]:
print(df_meth)

              cgID  GTEX-117YW-0003-SM-GW185  GTEX-11DYG-0004-SM-GW225  \
6       cg00000292                  0.946802                  0.884091   
75      cg00002426                  0.984741                  0.989391   
134     cg00003994                  0.040269                  0.021675   
268     cg00007981                  0.009185                  0.010464   
286     cg00008493                  0.979224                  0.990067   
...            ...                       ...                       ...   
753796  cg27654142                  0.142665                  0.066601   
753879  cg27657283                  0.015110                  0.013078   
754018  cg27662379                  0.015313                  0.022490   
754038  cg27662877                  0.011713                  0.017662   
754102  cg27665659                  0.023415                  0.030227   

        GTEX-11EMC-0004-SM-GW1YF  GTEX-11TT1-0001-SM-GW21F  \
6                       0.767099                 

In [5]:
#split training and test data with even age distributions 

from sklearn.model_selection import train_test_split  # Import train_test_split function


training_df = pd.DataFrame()
training_anno_df = pd.DataFrame()
testing_dfs = {}
testing_anno_dfs = {}

first_column_meth = pd.DataFrame()
first_column_anno = pd.DataFrame()

# Track the age counts in the training set
age_counts_training = {}

for organ_type in organs:
    print(f"Processing organ: {organ_type}")
    start_time = time.time()
    
    # Load the DataFrame
    df_meth = pd.read_table(f'/Users/yashagarwal/Documents/GTEx/GTEx_methylation/GTEx_{organ_type}.meth.csv')
    df_meth = df_meth[df_meth.iloc[:, 0].isin(list_cpgs)]
    df_meth = pd.DataFrame([df_meth.columns] + df_meth.values.tolist())

    df_anno = pd.read_table(f'/Users/yashagarwal/Documents/GTEx/GTEx_anno/GTEx_{organ_type}.anno.csv')
    age_sample_df = get_true_age_sample(df_anno)
    #print(df_anno.index)
    

    # Get the first column (index 0) from df_meth and df_anno
    first_column_meth = df_meth.iloc[:, 0]
    first_column_anno = df_anno.iloc[:, 0]
    #print(first_column_anno)

    # Get ages and sample IDs
    age_sample_df = get_true_age_sample(df_anno)
    
    # Group by age
    grouped = age_sample_df.groupby(age_sample_df.columns[1])  # Group by 'Age' column (index 1)

    train_samples_list = []
    test_samples_list = []

    for age, group in grouped:
        n_samples = len(group)
        n_train = int(0.8 * n_samples)
        
        # Ensure equal distribution across all organs
        if age in age_counts_training:
            max_train_samples = min(n_train, age_counts_training[age])
        else:
            max_train_samples = n_train
        
        if n_samples > 1:
            train, test = train_test_split(group, train_size=max_train_samples, random_state=42)
        else:
            train = group
            test = pd.DataFrame()  # No samples left for testing if only one sample exists
        
        # Update the age count in the training set
        if age in age_counts_training:
            age_counts_training[age] += len(train)
        else:
            age_counts_training[age] = len(train)
        
        train_samples_list.append(train)
        test_samples_list.append(test)

    
    # Concatenate all the training and testing samples together
    train_samples_df = pd.concat(train_samples_list)
    test_samples_df = pd.concat(test_samples_list)
    #print(train_samples_df)
    #print(train_samples_df.shape)

    # Split the data based on training and testing sample IDs
    train_sample_ids = train_samples_df.iloc[:, 0].values  # First column (sample IDs) index 0
    test_sample_ids = test_samples_df.iloc[:, 0].values   # First column (sample IDs) index 0
    
    #print(len(train_sample_ids))
    #print(len(test_sample_ids))
    
    # Filter df_meth based on sample IDs
    # Include the first column (sample IDs) and only columns whose IDs are in train_sample_ids
    # Move column names to first row and shift data down
    #df_meth.columns = pd.Index([''] * len(df_meth.columns))  # Create an empty row with column names
    
    
    
    training_columns_to_include = np.concatenate([[True], df_meth.iloc[0, 1:].isin(train_sample_ids)])
    train_meth_df = df_meth.iloc[:, training_columns_to_include]
    #print(df_meth.columns.shape)
    #print(df_meth.columns[1:].shape)
    #print(train_meth_df.shape)
    
    #print(train_meth_df)
    
    testing_columns_to_include = np.concatenate([[True], df_meth.iloc[0, 1:].isin(test_sample_ids)])
    #print(df_meth)
    test_meth_df = df_meth.iloc[:, testing_columns_to_include]

    #print(test_meth_df)

    #print(df_anno.iloc[0, 1:])  # Check the initial state of df_anno

    for col in range(1, len(df_anno.columns)):
        df_anno.iloc[9, col] = int(df_anno.iloc[9, col].split('-')[0]) + 5
        
    df_anno = df_anno.iloc[[9]]
    
    
    #print(df_anno.columns)
    train_anno_columns_to_include = np.concatenate([[True], df_anno.columns[1:].isin(train_sample_ids)])
    train_anno_df = df_anno.iloc[:, train_anno_columns_to_include]
    #print(train_anno_df.shape)
    #print(train_anno_df)
    
    
    test_anno_columns_to_include = np.concatenate([[True], df_anno.columns[1:].isin(test_sample_ids)])
    test_anno_df = df_anno.iloc[:, test_anno_columns_to_include]
    
    #print(test_anno_df)
    
    

    #print(train_meth_df.iloc[:, 1:])
    # Append to the training_df
    #print(training_df.shape)
    train_anno_df.reset_index(drop=True, inplace=True)
    test_anno_df.reset_index(drop=True, inplace=True)
    
    
    training_df = pd.concat([training_df, train_meth_df.iloc[:, 1:]], axis=1)
    #print(train_meth_df.iloc[:, 1:].shape)
    print("1")
    #print(training_df)
    print("2")
    training_anno_df = pd.concat([training_anno_df, train_anno_df.iloc[0:, 1:]], axis=1)
    #print(train_anno_df.iloc[1:, 1:])
    #print(training_anno_df)
    print(training_anno_df)
    
    # Store the testing DataFrames in the dictionaries
    testing_dfs[organ_type] = test_meth_df
    testing_anno_dfs[organ_type] = pd.DataFrame(test_anno_df.iloc[0:, 1:])
    
    print(f"Finished processing organ: {organ_type}")

# Combine first column back to training DataFrames
training_df = pd.concat([first_column_meth, training_df], axis=1)
#training_anno_df = pd.concat([first_column_anno, training_anno_df], axis=1)




Processing organ: Blood
1
2
  GTEX-11EMC-0004-SM-GW1YF GTEX-11TT1-0001-SM-GW21F GTEX-11ZU8-0002-SM-GW18V  \
0                       65                       25                       65   

  GTEX-12WSA-0001-SM-GW187 GTEX-12WSI-0004-SM-GW2CP GTEX-12ZZW-0003-SM-GW18R  \
0                       65                       55                       55   

  GTEX-12ZZY-0004-SM-GW19K GTEX-139YR-0004-SM-GW1DM GTEX-13NYS-0003-SM-GW1FJ  \
0                       65                       55                       65   

  GTEX-13NZA-0003-SM-GW1F3  ... GTEX-WFG8-0004-SM-GW21G  \
0                       65  ...                      25   

  GTEX-WWYW-0004-SM-GW17Q GTEX-X4EP-0002-SM-GW17R GTEX-X4XX-0004-SM-GW17N  \
0                      55                      65                      65   

  GTEX-X585-0003-SM-GW17P GTEX-X5EB-0001-SM-GW17Y GTEX-XPT6-0004-SM-GW1DN  \
0                      55                      45                      25   

  GTEX-YFC4-0001-SM-GW1YL GTEX-YFCO-0001-SM-GW29R GTEX-ZAB4-

In [ ]:
#split training and test data randomly 80%

import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from pathos.multiprocessing import ProcessingPool as Pool


import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import linregress
import time

# Define the organs list and initialize training and testing DataFrames

training_df = pd.DataFrame()
training_anno_df = pd.DataFrame()

# Dictionaries to store testing DataFrames by organ
testing_dfs = {}
testing_anno_dfs = {}
first_column_meth = pd.DataFrame()
first_column_anno = pd.DataFrame()
for organ_type in organs:
    print(f"Processing organ: {organ_type}")
    start_time = time.time()
    
    # Load the DataFrame
    df_meth = pd.read_table(f'/Users/yashagarwal/Documents/GTEx/GTEx_methylation/GTEx_{organ_type}.meth.csv')
    
    df_meth = df_meth[df_meth.iloc[:, 0].isin(list_cpgs)]
    #df_meth = impute_cpgs_df(df_meth)
    df_anno = pd.read_table(f'/Users/yashagarwal/Documents/GTEx/GTEx_anno/GTEx_{organ_type}.anno.csv')

    real_ages = get_true_age_sample(df_anno)
    print(real_ages)
    
    # Get the first column (index 0) from df_meth and df_anno
    first_column_meth = df_meth.iloc[:, 0]
    #print(first_column_meth)
    first_column_anno = df_anno.iloc[:, 0]

    # Get the remaining columns (from index 1 onward) from df_meth and df_anno
    remaining_columns_meth = df_meth.iloc[:, 1:]
    remaining_columns_anno = df_anno.iloc[:, 1:]

    # Calculate the split index
    split_index = int(np.floor(0.8 * remaining_columns_meth.shape[1]))

    # Get the column indexes
    train_indexes = remaining_columns_meth.columns[:split_index]
    test_indexes = remaining_columns_meth.columns[split_index:]

    # Split the remaining columns into training and testing sets for df_meth
    train_columns_meth = remaining_columns_meth[train_indexes]
    test_columns_meth = remaining_columns_meth[test_indexes]

    # Split the remaining columns into training and testing sets for df_anno
    train_columns_anno = remaining_columns_anno[train_indexes]
    test_columns_anno = remaining_columns_anno[test_indexes]

    # Add the first column back to each split of df_meth
    train_meth_df = pd.concat([first_column_meth, train_columns_meth], axis=1)
    test_meth_df = pd.concat([first_column_meth, test_columns_meth], axis=1)

    # Add the first column back to each split of df_anno
    train_anno_df = pd.concat([first_column_anno, train_columns_anno], axis=1)
    test_anno_df = pd.concat([first_column_anno, test_columns_anno], axis=1)

    print(train_meth_df.shape)
    # Append to the training_df
    training_df = pd.concat([training_df, train_meth_df.iloc[:, 1:]], axis=1)
    training_anno_df = pd.concat([training_anno_df, train_anno_df.iloc[:, 1:]], axis=1)
    print(training_anno_df)
    print(training_df.shape)
    # Store the testing DataFrames in the dictionaries
    testing_dfs[organ_type] = test_meth_df
    testing_anno_dfs[organ_type] = test_anno_df
    
    print(f"Finished processing organ: {organ_type} in {time.time() - start_time:.2f} seconds")

training_df = pd.concat([first_column_meth, training_df], axis=1)
training_anno_df = pd.concat([first_column_anno, training_anno_df], axis=1)

In [4]:
print(training_df.shape)
#training_df.to_csv('/Users/yashagarwal/Documents/training_df.csv', index=False)
print(training_anno_df.shape)


(21644, 42)
(1, 42)


In [152]:
training_df = training_df.to_csv("/Users/yashagarwal/Documents/new)training_meth_df.csv", index=False)
training_anno_df = training_anno_df.to_csv("/Users/yashagarwal/Documents/new_training__anno_df.csv", index=False)


In [224]:

training_df = pd.read_csv("/Users/yashagarwal/Documents/new)training_meth_df.csv")
training_anno_df = pd.read_csv("/Users/yashagarwal/Documents/new_training__anno_df.csv")
print(training_df.shape)
print(training_anno_df.shape)


/var/folders/sm/g8k8syc12ml4_ltnbwsxk8_m0000gn/T/ipykernel_47923/1803412936.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251

(21644, 652)
(1, 651)


In [6]:




'''file_list = ['tools/altum_cpgs_list.txt', 'tools/hannum_cpgs_list.txt', 'tools/horvath_cpgs_list.txt', 'tools/pheno_cpgs_list.txt', 'tools/horvath_SB_cpgs_list.txt', 'tools/han_cpgs_list.txt', 'tools/ying_cpgs_list.txt', 'tools/zhang_cpgs_list.txt']
    # Create an empty list to store the overlapping values
overlapping_cpgs = []
all_missing_values = []
# Iterate through the files in file_list
for file in file_list:
    count = 0
    with open(file, 'r') as file:
        cpgs_list = file.readlines()

    cpgs_list = [line.strip() for line in cpgs_list]
    
    missing_values = set(cpgs_list) - set(training_df.iloc[:, 0])
    all_missing_values = all_missing_values + list(missing_values)
all_missing_values = list(set(all_missing_values))
for value in all_missing_values:
    new_row = [value] + [np.nan] * (training_df.shape[1] - 1)  # Create a new row with value and 0.5 in other columns
    # print(new_row)
    training_df.loc[len(df)] = new_row  # Add new row to DataFrame

labels = training_df.iloc[:, 0]  # Assuming the first column contains the CPG labels
methylation_data = training_df.iloc[:, 1:]  # Assuming the methylation data starts from the second column

training_df.fillna(np.nan, inplace=True)

# Impute missing values using IterativeImputer
imputer_iterative = IterativeImputer(max_iter=10, random_state=0)  # You can adjust max_iter as needed
imputed_data_iterative = imputer_iterative.fit_transform(methylation_data)

# Convert imputed data back to DataFrame
# Create the DataFrame
imputed_df_iterative = pd.DataFrame(imputed_data_iterative, columns=methylation_data.columns)

# Add the labels as the first column
imputed_df_iterative.insert(0, 'Labels', labels)

training_df = imputed_df_iterative
print(training_df)'''

    
df = training_df

df.fillna(0.5, inplace=True)
    #print(df)
    #print(cpgs_list)
# Create new rows for missing values
missing_values = set(list_cpgs) - set(df.iloc[:, 0])

if missing_values:
    new_rows = pd.DataFrame([[value] + [0.5] * (df.shape[1] - 1) for value in missing_values], columns=df.columns)

    # Append new rows to DataFrame
    df = pd.concat([df, new_rows], ignore_index=True)

df = df.loc[~df.iloc[:, 0].duplicated(keep='first')]
training_df = df
print(training_df)


               0                         3                         4   \
0            cgID  GTEX-11EMC-0004-SM-GW1YF  GTEX-11TT1-0001-SM-GW21F   
1      cg00000292                  0.767099                  0.848465   
2      cg00002426                  0.976963                  0.959068   
3      cg00003994                  0.051398                  0.027254   
4      cg00007981                  0.013045                  0.005635   
...           ...                       ...                       ...   
22612  cg18202456                       0.5                       0.5   
22613  cg14073590                       0.5                       0.5   
22614  cg25291250                       0.5                       0.5   
22615  cg24110177                       0.5                       0.5   
22616  cg13735974                       0.5                       0.5   

                             5                         8   \
0      GTEX-11ZU8-0002-SM-GW18V  GTEX-12WSA-0001-SM-GW187   
1

In [7]:
training_df.columns = df.iloc[0]
training_df = training_df.iloc[0:]

# Drop the first row after setting it as the index
training_df = training_df.iloc[1:]
#trainin_df = training_df.apply(pd.to_numeric, errors='coerce')

print(training_df)

#predictions = pd.DataFrame({
 #   'horvath_age': get_horvath_sb_age(training_df)})
#print(predictions)


0            cgID GTEX-11EMC-0004-SM-GW1YF GTEX-11TT1-0001-SM-GW21F  \
1      cg00000292                 0.767099                 0.848465   
2      cg00002426                 0.976963                 0.959068   
3      cg00003994                 0.051398                 0.027254   
4      cg00007981                 0.013045                 0.005635   
5      cg00008493                 0.969187                 0.982755   
...           ...                      ...                      ...   
22612  cg18202456                      0.5                      0.5   
22613  cg14073590                      0.5                      0.5   
22614  cg25291250                      0.5                      0.5   
22615  cg24110177                      0.5                      0.5   
22616  cg13735974                      0.5                      0.5   

0     GTEX-11ZU8-0002-SM-GW18V GTEX-12WSA-0001-SM-GW187  \
1                     0.924429                 0.881642   
2                     0.98805

In [8]:
# Define a function to convert float to int if it's a float
def convert_float_to_int(value):
    if isinstance(value, float):
        return int(value)
    else:
        return value

# Apply the function to each cell in the DataFrame
training_df = training_df.applymap(convert_float_to_int)
print(training_df)

# Assuming df is your DataFrame
first_row = training_df.iloc[2615]  # Selecting the first row

# Checking the data types of values in the first row
float_count = sum(isinstance(value, float) for value in first_row)

print(f"Number of float values in the first row: {float_count}")

row_values = training_df.iloc[2615]

float_values = [value for value in row_values if isinstance(value, float)]

print(f"Float values in the 23rd row: {float_values}")



/var/folders/sm/g8k8syc12ml4_ltnbwsxk8_m0000gn/T/ipykernel_55561/3948483316.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  training_df = training_df.applymap(convert_float_to_int)


0            cgID  GTEX-11EMC-0004-SM-GW1YF  GTEX-11TT1-0001-SM-GW21F  \
1      cg00000292                         0                         0   
2      cg00002426                         0                         0   
3      cg00003994                         0                         0   
4      cg00007981                         0                         0   
5      cg00008493                         0                         0   
...           ...                       ...                       ...   
22612  cg18202456                         0                         0   
22613  cg14073590                         0                         0   
22614  cg25291250                         0                         0   
22615  cg24110177                         0                         0   
22616  cg13735974                         0                         0   

0      GTEX-11ZU8-0002-SM-GW18V  GTEX-12WSA-0001-SM-GW187  \
1                             0                         0   
2

In [9]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge




# Get the age predictions
predictions = pd.DataFrame({
    'altum_age': get_altum_age(training_df),
    'horvath_age': get_horvath_age(training_df),
    'hannum_age': get_hannum_age(training_df),
    'pheno_age': get_pheno_age(training_df),
    'horvath_sb_age': get_horvath_sb_age(training_df),
    'ZhangEn_age': get_ZhangEn_age(training_df),
    'Han2020_age': get_Han2020_age(training_df),
    'YingCaus_age': get_YingCaus_age(training_df)
})
#print(predictions.shape)

real_ages = get_true_age(training_anno_df)

pca = PCA(n_components='mle')


# Perform GBDT
X = predictions.values
y = real_ages

#X_pca = pca.fit_transform(X)

#model = GradientBoostingRegressor()
#model = RandomForestRegressor()
#model = ElasticNet()
#model = Ridge()
#model = LinearRegression()
#model.fit(X_pca, y)

#model = Lasso()
model = BayesianRidge()

model.fit(X, y)
coefficients = model.coef_

print(coefficients)

# The model will assign weights to each clock based on their importance
#feature_importances = model.feature_importances_

#print(model.feature_importances_)

/Users/yashagarwal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(


21/21 [==============================] - 1s 4ms/step


/Users/yashagarwal/Documents/GitHub/EnsembleMeAgingClock/app.py:170: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred = np.matmul (input2, coef) + df_slope_only['CoefficientTraining'][0]


[34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242 34.60565242
 34.60565242 34.60565242 34.60565242 34.60565242 34

IndexError: single positional indexer is out-of-bounds

In [239]:
# Function to apply weighted predictions to a testing DataFrame
def apply_weighted_predictions(df):
    preds = np.array([
        get_altum_age(df),
        get_horvath_age(df),
        get_hannum_age(df),
        get_pheno_age(df),
        get_horvath_sb_age(df),
        get_ZhangEn_age(df),
        get_Han2020_age(df),
        get_YingCaus_age(df)
    ])
    weighted_preds = np.dot(preds.T, coefficients)
    return weighted_preds



In [240]:
for organ_type in organs:
    df = testing_dfs[organ_type]
    df.fillna(0.5, inplace=True)
        #print(df)
        #print(cpgs_list)
    # Create new rows for missing values
    missing_values = set(list_cpgs) - set(df.iloc[:, 0])

    if missing_values:
        new_rows = pd.DataFrame([[value] + [0.5] * (df.shape[1] - 1) for value in missing_values], columns=df.columns)

        # Append new rows to DataFrame
        df = pd.concat([df, new_rows], ignore_index=True)

    df = df.loc[~df.iloc[:, 0].duplicated(keep='first')]
    testing_dfs[organ_type] = df

KeyError: 'Blood'

In [39]:
import numpy as np
from scipy.stats import linregress

results = []

for organ_type in organs:
    test_df = testing_dfs[organ_type]
    # test_df = impute_cpgs_df(test_df)
    
    test_anno_df = testing_anno_dfs[organ_type]
    true_ages = get_true_age(test_anno_df)
    
    weighted_predictions = apply_weighted_predictions(test_df)
    mean_predictions = np.mean([get_altum_age(test_df), get_horvath_age(test_df), get_hannum_age(test_df), get_pheno_age(test_df), get_horvath_sb_age(test_df)], axis=0)
    
    # Calculate the slope of correlation between weighted predictions and true ages
    slope_weighted, _, _, _, _ = linregress(weighted_predictions, true_ages)
    slope_mean, _, _, _, _ = linregress(mean_predictions, true_ages)
    
    results.append(f'With weighted clock, Slope of correlation for {organ_type}: {slope_weighted}')
    results.append(f'With mean of clocks, Slope of correlation for {organ_type}: {slope_mean}')

for result in results:
    print(result)

/var/folders/sm/g8k8syc12ml4_ltnbwsxk8_m0000gn/T/ipykernel_39259/1623059914.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ages.append(int(age_data[i].split('-')[0]) + 5)
/Users/yashagarwal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 0s 26ms/step
[45.54246606 44.50826102 44.9988647  67.29994056 54.19840059 41.86536864
 36.55423012 11.81877431 36.89638501 43.2084483  41.81766399]
<class 'numpy.ndarray'>
70
770
coef: (513,)
meth: (513, 11)


/Users/yashagarwal/Documents/GitHub/EnsembleMeAgingClock/app.py:171: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred = np.matmul (input2, coef) + df_slope_only['CoefficientTraining'][0]
/Users/yashagarwal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 0s 25ms/step
[45.54246606 44.50826102 44.9988647  67.29994056 54.19840059 41.86536864
 36.55423012 11.81877431 36.89638501 43.2084483  41.81766399]
<class 'numpy.ndarray'>
70
770
coef: (513,)
meth: (513, 11)


/Users/yashagarwal/Documents/GitHub/EnsembleMeAgingClock/app.py:171: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred = np.matmul (input2, coef) + df_slope_only['CoefficientTraining'][0]
/var/folders/sm/g8k8syc12ml4_ltnbwsxk8_m0000gn/T/ipykernel_39259/1623059914.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ages.append(int(age_data[i].split('-')[0]) + 5)
/Users/yashagarwal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(


...

/Users/yashagarwal/Documents/GitHub/EnsembleMeAgingClock/app.py:171: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred = np.matmul (input2, coef) + df_slope_only['CoefficientTraining'][0]
/var/folders/sm/g8k8syc12ml4_ltnbwsxk8_m0000gn/T/ipykernel_39259/1623059914.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ages.append(int(age_data[i].split('-')[0]) + 5)
/Users/yashagarwal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 0s 22ms/step
[33.09606142 36.87781727 29.79440035 30.57201803 26.75322857 44.51815942
 39.51048484 47.17091307 39.65702078 33.70385272]
<class 'numpy.ndarray'>
70
700
coef: (513,)
meth: (513, 10)


/Users/yashagarwal/Documents/GitHub/EnsembleMeAgingClock/app.py:171: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred = np.matmul (input2, coef) + df_slope_only['CoefficientTraining'][0]
/Users/yashagarwal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 0s 22ms/step
[33.09606142 36.87781727 29.79440035 30.57201803 26.75322857 44.51815942
 39.51048484 47.17091307 39.65702078 33.70385272]
<class 'numpy.ndarray'>
70
700
coef: (513,)
meth: (513, 10)
With weighted clock, Slope of correlation for Blood: 1.1529504015467775
With mean of clocks, Slope of correlation for Blood: 0.7303128974114547
With weighted clock, Slope of correlation for Breast: 0.7435490428041965
With mean of clocks, Slope of correlation for Breast: 0.9176664995639522
With weighted clock, Slope of correlation for Colon: 1.1289626796740933
With mean of clocks, Slope of correlation for Colon: 0.4751348070436736
With weighted clock, Slope of correlation for Kidney: 1.409929307010253
With mean of clocks, Slope of correlation for Kidney: 0.7307444415832473
With weighted clock, Slope of correlation for Lung: 1.202440090617954
With mean of clocks, Slope of correlation for Lung: 0.9569116639259825
With weighted clock, Slope of correlation for

/Users/yashagarwal/Documents/GitHub/EnsembleMeAgingClock/app.py:171: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred = np.matmul (input2, coef) + df_slope_only['CoefficientTraining'][0]


In [ ]:
for organ_type in organs:
        df = pd.read_table('/Users/yashagarwal/Documents/GTEx/GTEx_methylation/GTEx_' + organ_type + '.meth.csv')
        #df = impute_cpgs_df(df)
        print(df)
        
        real_age = get_true_age('input/GTEx/GTEx_anno_GTEx_' + organ_type + '.anno.csv')
        
        #print('Ensemble Naive')
        mean_pred = np.mean([get_altum_age(df), get_horvath_age(df), get_hannum_age(df), get_pheno_age(df), get_horvath_sb_age(df), get_ZhangEn_age(df), get_Han2020_age(df), get_YingCaus_age(df)], axis=0)
        #print(pred)